In [1]:
%run ComputeRateFromParams.ipynb

Generating cache table of size 512x512x128 (~25.0 bits)...


  0%|          | 0/511 [00:00<?, ?it/s]

Done. Table generation took 1.6 seconds


In [2]:
from multiprocessing import Pool

In [3]:
def compute_func_and_jacobian(f, p: tuple):
    """
    Given some function f and a point p, computes the value of f at p and estimates its Jacobian by looking at p + epsilon e_i.
    """
    foo = lambda p: f(*p)
    DELTA = 1E-10
    inputs = [p] + [p[:i] + (p[i] + DELTA,) + p[i+1:] for i in range(len(p))]
    with Pool(len(p) + 1) as pool:
        points = list(pool.map(foo, inputs))
    base_point = points[0]
    derivatives = np.zeros(len(p))
    for i in range(len(p)):
        derivatives[i] = (points[i+1] - base_point) / DELTA
    return (base_point, derivatives)

def get_direction(jacobian: np.ndarray):
    return jacobian / np.linalg.norm(jacobian)

def get_second_derivative(f, f_at_p: float, p: tuple, d: np.ndarray):
    '''
    Given a function f, a starting point p, the value of f at p and a direction d, 
        computes the first and second derivatives of f along d.
    '''
    DELTA = 1E-6
    foo = lambda p: f(*p)
    inputs = [[[p[i] + DELTA*d[i] for i in range(len(p))]], [p[i] - DELTA*d[i] for i in range(len(p))]]
    with Pool(len(p) + 1) as pool:
        points = list(pool.map(foo, inputs))
    plus_delta = points[0]
    minus_delta = points[1]
    first_derivative = (plus_delta - minus_delta) / (2 * DELTA)
    second_derivative = (plus_delta + minus_delta - (2 * f_at_p)) / (DELTA ** 2)
    return (first_derivative, second_derivative)
    

def do_optimization_step(f, current_point: tuple, jump_limit: float = 0.1):
    '''
    Given a function f, a starting point and an upper bound on the distance of the jump allowed,
        performs a step of gradient descent to maximize the function f.
    Will ensure that all entries of the output point and all intermediary points are positive.
    Returns as tuple of:
        - The next point
        - The value of f at the starting point
        - The jacobian of f at the starting point
    '''
    bp, jac = compute_func_and_jacobian(f, current_point)
    d = get_direction(jac)
    first_derivative, second_derivative = get_second_derivative(f, bp, current_point, d)
    jump_size = first_derivative / (-(second_derivative))
    jump_size = min(jump_size, jump_limit)
    if jump_size < 0:
        jump_size = jump_limit
    for i in range(len(current_point)):
        if current_point[i] + (jump_size * d[i]) < 0:
            jump_size = min(jump_size, -current_point[i] / (2 * d))
    print(jump_size, jump_size * np.dot(d, jac))
    next_point = tuple([current_point[i] + (jump_size * d[i]) for i in range(len(p))])
    return (next_point, bp, jac)

In [4]:
p = (1.79, 0.1, 0.23, 0.1)
t0 = time.time()
data = []
for i in it.count():
    p, bp, jac = do_optimization_step(compute_rate_from_beta_g, p, 0.025)
    data.append((p, bp, jac))
    print(p, bp, '%.1f' % (time.time() - t0))

AttributeError: Can't pickle local object 'compute_func_and_jacobian.<locals>.<lambda>'

In [ ]:
p =(1.79, 0.1, 0.23, 0.1)
bp, jac = compute_func_and_jacobian(compute_rate_from_beta_g, p)

In [ ]:
d = get_direction(jac)

In [ ]:
get_second_derivative(compute_rate_from_beta_g, bp, p, d)

In [ ]:
np.dot(d, jac)

In [ ]:
np.dot((-0.5 * (_13[0] / _13[1]) * d), jac)

In [ ]:
bp